In [ ]:
! curl -sSL https://install.python-poetry.org | python3 -

Retrieving Poetry metadata

# Welcome to Poetry!

This will download and install the latest version of Poetry,
a dependency and package manager for Python.

It will add the `poetry` command to Poetry's bin directory, located at:

/root/.local/bin

You can uninstall at any time by executing this script with the --uninstall option,
and these changes will be reverted.

Installing Poetry (2.1.2)
Installing Poetry (2.1.2): Creating environment
Installing Poetry (2.1.2): Installing Poetry
Installing Poetry (2.1.2): Creating script
Installing Poetry (2.1.2): Done

Poetry (2.1.2) is installed now. Great!

To get started you need Poetry's bin directory (/root/.local/bin) in your `PATH`
environment variable.

Add `export PATH="/root/.local/bin:$PATH"` to your shell configuration file.

Alternatively, you can call Poetry explicitly with `/root/.local/bin/poetry`.

You can test that everything is set up by executing:

`poetry --version`



In [ ]:
! git clone https://github.com/mrapplexz/nn-misis-2526

Cloning into 'nn-misis-2526'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 133 (delta 47), reused 112 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (133/133), 1.04 MiB | 15.87 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [ ]:
! cd nn-misis-2526; git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 1.58 KiB | 134.00 KiB/s, done.
From https://github.com/mrapplexz/nn-misis-2526
   ad12e1b..0cf1a69  master     -> origin/master
Updating ad12e1b..0cf1a69
Fast-forward
 poetry.lock    | 31 ++++++++++++++++++++++++++++++-
 pyproject.toml |  1 +
 2 files changed, 31 insertions(+), 1 deletion(-)


In [ ]:
! cd nn-misis-2526; /root/.local/bin/poetry install

Installing dependencies from lock file

Package operations: 2 installs, 0 updates, 0 removals

  - Installing traitlets (5.14.3): Pending...
  - Installing traitlets (5.14.3): Downloading... 0%
  - Installing traitlets (5.14.3): Downloading... 100%
  - Installing traitlets (5.14.3): Installing...
  - Installing traitlets (5.14.3)
  - Installing matplotlib-inline (0.1.7): Pending...
  - Installing matplotlib-inline (0.1.7): Downloading... 0%
  - Installing matplotlib-inline (0.1.7): Downloading... 100%
  - Installing matplotlib-inline (0.1.7): Installing...
  - Installing matplotlib-inline (0.1.7)

Installing the current project: misis-2526 (0.1.0)
Error: The current project could not be installed: Readme path `/content/nn-misis-2526/README.md` does not exist.
If you do not want to install the current project use --no-root.
If you want to use Poetry only for dependency management but not for packaging, you can disable package mode by setting package-mode = false in your pyproject.toml f

In [ ]:
! cd nn-misis-2526; /root/.local/bin/poetry run accelerate launch -m misisnn.retrieval.train --config-path configs/retrieval.json

In [ ]:
! cd nn-misis-2526; /root/.local/bin/poetry run python -m misisnn.retrieval.create_faiss_index

Usage: python -m misisnn.retrieval.create_faiss_index [OPTIONS]
Try 'python -m misisnn.retrieval.create_faiss_index --help' for help.

Error: Missing option '--config-path'.


In [ ]:
! pip install -U huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.3/509.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 83.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.31.2
    Uninstalling huggingface-hub-0.31.2:
      Successfully uninstalled huggingface-hub-0.31.2


In [1]:
import numpy as np
from typing import List

def recall_at_k(targets: List[str],
                predictions: List[List[str]],
                k: int = 10) -> float:
    targets = np.array(targets)
    hits = np.array([np.isin(target, pred[:k]) for target, pred in zip(targets, predictions)])
    return float(hits.mean())


def mean_reciprocal_rank(targets: List[str],
                        predictions: List[List[str]]) -> float:
    targets = np.array(targets)
    ranks = []
    for target, pred in zip(targets, predictions):
        if target in pred:
            ranks.append(1 / (pred.index(target) + 1))
        else:
            ranks.append(0)
    return float(np.mean(ranks))


In [2]:
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# Load dataset directly from Hugging Face
dataset = load_dataset("sentence-transformers/natural-questions")
full_train_df = pd.DataFrame(dataset['train'])

# Split into train and test sets
train_data, test_data = train_test_split(full_train_df, test_size=0.2, random_state=42)
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

corpus = pd.concat([train_data['query'], train_data['answer']]).tolist()
vectorizer = TfidfVectorizer(stop_words='english')
vectorizer.fit(corpus)

test_queries = vectorizer.transform(test_data['query'])
test_answers = vectorizer.transform(test_data['answer'])

cosine_sim = cosine_similarity(test_queries, test_answers)

predictions = []
for scores in cosine_sim:
    top_indices = scores.argsort()[::-1][:10]
    predictions.append(test_data['answer'].iloc[top_indices].tolist())ё

print(f"Recall@1: {recall_at_k(test_data['answer'].tolist(), predictions, 1):.4f}")
print(f"Recall@3: {recall_at_k(test_data['answer'].tolist(), predictions, 3):.4f}")
print(f"Recall@10: {recall_at_k(test_data['answer'].tolist(), predictions, 10):.4f}")
print(f"MRR: {mean_reciprocal_rank(test_data['answer'].tolist(), predictions):.4f}")

Recall@1: 0.4323
Recall@3: 0.6120
Recall@10: 0.7711
MRR: 0.5402


Prediction: 100%|██████████| 20047/20047 [00:15<00:00, 1292.08it/s]


Recall@1: 0.7480
Recall@3: 0.8976
Recall@10: 0.9665
MRR: 0.8284


In [4]:
from sentence_transformers import SentenceTransformer, losses
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, Subset
from typing import List, Tuple
from tqdm import tqdm
import random

class QuestionAnswerDataset(Dataset):
    def __init__(self, queries: List[str], answers: List[str]):
        self.queries = queries
        self.answers = answers

    def __len__(self) -> int:
        return len(self.queries)

    def __getitem__(self, idx: int) -> Tuple[str, str]:
        return self.queries[idx], self.answers[idx]


def train_epoch(
    model: SentenceTransformer,
    dataloader: DataLoader,
    loss_fn: losses.MultipleNegativesRankingLoss,
    optimizer: torch.optim.Optimizer,
    device: torch.device
) -> float:
    model.train()
    total_loss = 0

    for queries, answers in tqdm(dataloader, desc='Training'):
        # Tokenize queries and answers separately
        query_features = model.tokenize(queries)
        answer_features = model.tokenize(answers)

        # Move features to device
        query_features = {k: v.to(device) for k, v in query_features.items()}
        answer_features = {k: v.to(device) for k, v in answer_features.items()}

        # Get embeddings
        query_emb = model(query_features)['sentence_embedding']
        answer_emb = model(answer_features)['sentence_embedding']

        loss = loss_fn(query_emb, answer_emb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)


def evaluate(
    model: SentenceTransformer,
    test_data: pd.DataFrame,
    device: torch.device,
    batch_size: int = 32
) -> None:
    model.eval()
    all_predictions = []
    all_targets = test_data['answer'].tolist()

    with torch.no_grad():
        for i in range(0, 3, batch_size):
            batch_queries = test_data['query'].iloc[i:i + batch_size].tolist()
            batch_answers = test_data['answer'].iloc[i:i + batch_size].tolist()

            query_emb = model.encode(batch_queries, convert_to_tensor=True, device=device)
            answer_emb = model.encode(batch_answers, convert_to_tensor=True, device=device)

            similarities = torch.nn.functional.cosine_similarity(
                query_emb.unsqueeze(1),
                answer_emb.unsqueeze(0),
                dim=2
            )

            for scores in similarities:
                top_indices = scores.argsort(descending=True)[:10]
                predictions = [batch_answers[idx] for idx in top_indices]
                all_predictions.append(predictions)

    metrics = {
        'Recall@1': recall_at_k(all_targets, all_predictions, 1),
        'Recall@3': recall_at_k(all_targets, all_predictions, 3),
        'Recall@10': recall_at_k(all_targets, all_predictions, 10),
        'MRR': mean_reciprocal_rank(all_targets, all_predictions)
    }

    for metric, value in metrics.items():
        print(f'{metric}: {value:.4f}')

# Create full dataset
full_dataset = QuestionAnswerDataset(train_data['query'].tolist(), train_data['answer'].tolist())

# Setup model and training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SentenceTransformer('intfloat/e5-base').to(device)
loss_fn = losses.MultipleNegativesRankingLoss(model)ё
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Training parameters
num_epochs = 100
samples_per_epoch = len(full_dataset) // 100
best_val_loss = float('inf')

for epoch in range(num_epochs):
    print(f'\nEpoch {epoch + 1}/{num_epochs}')

    # Create random subset for this epoch
    indices = random.sample(range(len(full_dataset)), samples_per_epoch)
    epoch_dataset = Subset(full_dataset, indices)
    train_dataloader = DataLoader(epoch_dataset, batch_size=32, shuffle=True)

    # Train
    train_loss = train_epoch(model, train_dataloader, loss_fn, optimizer, device)
    print(f'Training Loss: {train_loss:.4f}')

    # Evaluate
    print('Test Metrics:')
    evaluate(model, test_data, device)


Epoch 1/100


Training:   0%|          | 0/26 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 48.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 38.12 MiB is free. Process 545044 has 14.70 GiB memory in use. Of the allocated memory 14.57 GiB is allocated by PyTorch, and 8.30 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)